In [1]:
import filetype
from pathlib import Path
import nbimporter
import gradio as gr

In [2]:
def get_gradio_chat_launcher(chat):
    return gr.ChatInterface(fn=chat, type="messages")

In [3]:
def  get_new_chat_interface(chat):
    with gr.Blocks() as ui:
        with gr.Row():
            chatbot = gr.Chatbot(height=500, type='messages')
            image_output = gr.Image(height=500)
        with gr.Row():
            text_input = gr.Textbox(label='Chat with the AI assistant')
        with gr.Row():
            clear = gr.Button('Clear')

        def do_entry(message, history):
            history=history+[{'role':'user','content':message}]
            return '', history

        text_input.submit(do_entry, inputs=[text_input, chatbot], outputs=[text_input,chatbot]).then(
            chat, inputs=[chatbot,text_input],outputs=[chatbot,image_output]
        )
        clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)
    return ui
        

def get_gradio_launcher(func):
    view = gr.Interface(
        fn=func,
        inputs=[gr.Textbox(label="Your message:"),gr.File(label="Upload file"),gr.Dropdown(["GPT", "GEMINI"], label="Select model")],
        outputs=[gr.Markdown(label="Response:")],
        flagging_mode="never"
    )
    return view
    #view.launch()#view.launch(server_name="10.75.32.62", server_port=7866)

In [4]:
def get_gradio_multi_modal_launcher(func):
    view = gr.Interface(
        fn=func,
        inputs=[gr.Textbox(label="Your message:"),gr.File(label="Upload file"),gr.Dropdown(["GPT", "GEMINI"], label="Select model")],
        outputs=[gr.Markdown(label="Response:"),gr.Image(height=512)],
        flagging_mode="never"
    )
    return view

In [5]:
def is_ascii(file_path):
    try:
        with open(file_path, 'rb') as f:
            content = f.read()
            content.decode('ascii')  # Will raise UnicodeDecodeError if not ASCII
        return True
    except UnicodeDecodeError:
        return False
        
def get_file_path_str(file_path):
    if(isinstance(file_path,Path)):
        file_path=file_path.as_posix()
    return file_path 
    
def detect_mime_type(file_path):
    file_path=get_file_path_str(file_path)
    if is_ascii(file_path):
        return "text/plain"

    kind = filetype.guess(file_path)
    if kind:
        return kind.mime    

    return None

# Example usage
print(detect_mime_type("C:/Projects/llm_engg/llm_engineering/requirements.txt"))


text/plain


In [6]:
def getFileContent(fp):  
    fp=get_file_path_str(fp)
    with open(fp, 'r') as file:
        file_content = file.read()
        return file_content
    return ""

In [7]:
def append_conv(conv, msg):
    conv.append(msg)
    return conv

In [8]:
def create_gpt_prompt(qt, role):
    """
    Creates a correctly formatted Python dictionary for an OpenAI API message.
    """
    return {
        "role": role,
        "content": qt
    }

In [9]:
def create_gpt_tool_response_dict(tool, response):
    return {
        "tool_call_id" : tool.id,
        "role" : "tool",
        "name" : tool.function.name,
        "content" : response
    }

In [10]:
def reset_conv_history(conv,sys_msg):
    conv=[]
    conv.append(sys_msg)
    return conv

In [11]:
import inspect
from typing import get_origin, get_args, Literal
def create_tool_from_function(func):
    """
    Generates an OpenAI-compatible tool definition from a Python function.

    This function leverages type hints and a structured docstring to create 
    the JSON schema required by the OpenAI API for tool-calling.

    It supports multi-line function descriptions and multi-line parameter
    descriptions.

    Args:
        func (callable): The function to be converted into a tool.

    Returns:
        dict: A dictionary representing the tool in the format expected by
              the OpenAI API.
    """
    # Get the clean, un-indented docstring
    full_docstring = inspect.getdoc(func)
    if not full_docstring:
        raise ValueError("The function must have a docstring to be used as a tool.")
    
    lines = [line.strip() for line in full_docstring.strip().split('\n')]
    
    # --- 1. Parse Function and Parameter Descriptions ---
    try:
        args_section_index = lines.index('Args:')
    except ValueError:
        args_section_index = len(lines)

    func_description = "\n".join(lines[:args_section_index]).strip()
    
    # Robustly parse multi-line argument descriptions
    param_docs = {}
    current_param_name = None
    args_section_lines = lines[args_section_index + 1:]

    for line in args_section_lines:
        if ':' in line:
            param_name, param_desc = line.split(':', 1)
            param_name = param_name.split('(')[0].strip()
            # Start a new list of description lines for this parameter
            param_docs[param_name] = [param_desc.strip()]
            current_param_name = param_name
        elif current_param_name:
            # If a line doesn't have a colon, it's a continuation of the last parameter
            param_docs[current_param_name].append(line.strip())

    # Join the multi-line descriptions back into single strings
    for name, desc_lines in param_docs.items():
        param_docs[name] = "\n".join(desc_lines)

    # --- 2. Introspect Function Signature ---
    sig = inspect.signature(func)
    parameters_schema = {"type": "object", "properties": {}, "required": []}
    type_mapping = {str: "string", int: "integer", float: "number", bool: "boolean", dict: "object"}

    for name, param in sig.parameters.items():
        if param.default == inspect.Parameter.empty and name != 'self':
            parameters_schema["required"].append(name)

        param_info = {}
        if get_origin(param.annotation) is Literal:
            param_info["type"] = "string"
            param_info["enum"] = list(get_args(param.annotation))
        else:
            param_info["type"] = type_mapping.get(param.annotation, "string")
            
        if name in param_docs:
            param_info["description"] = param_docs[name]

        parameters_schema["properties"][name] = param_info
        
    if not parameters_schema["properties"]:
        parameters_schema = {"type": "object", "properties": {}}

    # --- 3. Assemble Final Tool ---
    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": func_description,
            "parameters": parameters_schema,
        },
    }

In [14]:
from google import genai
def get_list_of_models():
    client = genai.Client()
    
    print("List of models that support generateContent:\n")
    for m in client.models.list():
        for action in m.supported_actions:
            if action == "generateContent":
                print(m.name)
    print("List of models that support embedContent:\n")
    for m in client.models.list():
        for action in m.supported_actions:
            if action == "embedContent":
                print(m.name)